# 1. Import necessary libraries

In [1]:
import pandas as pd
import numpy as np
import re
import nltk
import spacy
import scipy
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_validate
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
from textblob import Word
from sklearn.utils import resample
from sklearn.neural_network import MLPClassifier
from sklearn.feature_extraction.text import TfidfVectorizer
from imblearn.over_sampling import SMOTE
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import cross_validate
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import f1_score,accuracy_score,precision_score,recall_score

In [2]:
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /Users/alina/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [3]:
!pip3 install spacy
!python3 -m spacy download en_core_web_sm

You should consider upgrading via the '/Users/alina/Desktop/NUS/semster4/CS5344/Project Handout and Readings/pythonProject/venv/bin/python -m pip install --upgrade pip' command.
     |████████████████████████████████| 13.9 MB 569 kB/s eta 0:00:01
You should consider upgrading via the '/Users/alina/Desktop/NUS/semster4/CS5344/Project Handout and Readings/pythonProject/venv/bin/python3 -m pip install --upgrade pip' command.
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_sm')


# 2. Read Data

In [4]:
#Read files
train = pd.read_csv("./original dataset/fulltrain.csv",header=None)
test = pd.read_csv("./original dataset/balancedtest.csv",header=None)

In [5]:
train.columns = ['Verdict','Text']
test.columns = ['Verdict','Text']

In [6]:
#Unbalance training data
train['Verdict'].value_counts()

3    17870
1    14047
4     9995
2     6942
Name: Verdict, dtype: int64

In [7]:
#Balance test data
test['Verdict'].value_counts()

1    750
2    750
3    750
4    750
Name: Verdict, dtype: int64

In [8]:
train1=train.copy()
test1 = test.copy()

In [9]:
from sklearn.model_selection import train_test_split
y=train['Verdict']
X=train['Text']
X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.1, random_state=42)

In [10]:
X_test_p = pd.DataFrame(X_test)
y_test_p = pd.DataFrame(y_test)

In [11]:
data_small = pd.concat([X_test_p,y_test_p],axis=1)
data_small

,Text,Verdict
14019,According to witnesses at your old high school...,1
44568,The National Immigration Agency (NIA) announce...,4
43613,The enforcement division of the Securities and...,4
2213,"In an effort to bolster its flagging ratings, ...",1
17804,Sarah Palin Makes Tragically Heartbreaking New...,2
...,...,...
3504,Back to story: Netflix Board Of Directors Meet...,1
28162,Germany Bans Fracking But Theres a HitchBy Bra...,3
43123,They are forever shocked to be visited in thei...,4
14471,WOW: Secret Hillary Video Leaks... She Just Lo...,2


In [12]:
data_small_X = data_small['Text']
data_small_y = data_small['Verdict']

In [13]:
test_X = test['Text']
test_y = test['Verdict']

# 3. Preprocessing for Text

In [14]:
def preprocess_text(s, replace=None, remove_punctuation = None, lower=None,stopword=None,frequency_words=None,scared_word=None, noisy=None, stemming=None,lemmatization=None):
    #Throw an error is both stemming and lemmatization are not None

    s1 = s.copy()
    if stemming is not None and lemmatization is not None:
        raise ValueError('Stemming and Lemmatization cannot both be not None!')


    if replace is not None:
        #Replace URLs with 'webaddress'
        s1['Text'] = s1['Text'].str.replace(r'^http\://[a-zA-Z0-9\-\.]+\.[a-zA-Z]{2,3}(/\S*)?$',
                                  'webaddress',regex=True)
        #Replace email address with 'email'
        s1['Text'] = s1['Text'].str.replace(r'^.+@[^\.].*\.[a-z]{2,}$',
                                 'emailaddress',regex=True)
        #Replace money symbols with 'moneysymb' (£ can by typed with ALT key + 156)
        s1['Text'] = s1['Text'].str.replace(r'£|\$', 'moneysymb',regex=True)

        #Replace percentage symbols with 'percentage'
        s1['Text'] = s1['Text'].str.replace(r'%', 'percentage',regex=True)

        #Replace 10 digit phone number
        s1['Text'] = s1['Text'].str.replace(r'^\(?[\d]{3}\)?[\s-]?[\d]{3}[\s-]?[\d]{4}$',
                                  'phonenumbr',regex=True)
        # Replace numbers with 'numbr'
        s1['Text'] = s1['Text'].str.replace(r'\d+(\.\d+)?', 'numbr',regex=True)
    #Remove punctuation
    if remove_punctuation is not None:
        s1['Text'] = s1['Text'].apply(lambda x: re.sub(r'[^\w\s\d]', '', x))

    #Transform to lower letter
    if lower is not None:
        s1['Text'] = s1['Text'].apply(lambda x: x.lower())

    #Remove the stopwords
    if stopword is not None:
        stop=stopwords.words('english')
        s1['Text']=s1['Text'].apply(lambda sen:" ".join(x for x in sen.split() if x not in stop))

    #Remove the frequency words
    if frequency_words is not None:
        freq=pd.Series(' '.join(s).split()).value_counts()[:10]
        freq=list(freq.index)
        s1['Text']=s1['Text'].apply(lambda sen:' '.join(x for x in sen.split() if x not in freq))

    # Remove the scarce word
    if scared_word is not None:
        scared = pd.Series(' '.join(s).split()).value_counts()[-10:]
        scared = list(scared.index)
        s1['Text'] = s1['Text'].apply(lambda sen: " ".join(x for x in sen.split() if x not in scared))

    #Noisy Removal
    if noisy is not None:
        #remove non-ascii
        s1['Text']= s1['Text'].apply(lambda x: re.sub("(\\W)"," ",x))
        #remove whitespace
        s1['Text']=s1['Text'].apply(lambda x: x.strip())

    #Stemming
    if stemming is not None:
        ps = PorterStemmer()
        s1['Text']=s1['Text'].apply(lambda x:" ".join(ps.stem(word) for word in x.split()))

    #Lemmatization
    if lemmatization is not None:
        nltk.download('wordnet')
        s1['Text']= s1['Text'].apply(lambda x:" ".join([Word(word).lemmatize() for word in x.split()]))

    return s1

# 4. Feature Engineering

In [15]:
#Combine three feature engineering methods into one class
def feature_engineering(s, train=None,tf_idf=None, word2vec=None, word_count=None):
    #1. TF-IDF
    s1 = s.copy()
    if tf_idf is not None:
        tfv = TfidfVectorizer(min_df=3,  max_features=None,strip_accents='unicode', analyzer='word',token_pattern=r'\w{1,}', ngram_range=(1, 3), use_idf=1,smooth_idf=1,sublinear_tf=1, stop_words = 'english')
        # Fitting TF-IDF to both training and test sets (semi-supervised learning)
        tfv.fit(list(train['Text']))
        features =  tfv.transform(s1['Text'])
    #2. Word2Vec
    if word2vec is not None:
        nlp = spacy.load('en_core_web_sm')
        features = []
        for sentence in s1['Text']:
            doc = nlp(sentence)
            features.append(doc.vector)
    #3. Word-count document
    if word_count is not None:
        #Instantiate the vectorizer
        count_vectorizer = CountVectorizer()
        features = count_vectorizer.fit_transform(s1['Text'])

    return features

# 5. Logistic Regression

## 5.1 Data Preprocessing (no data preprocessing vs data preprocessing)

In [16]:
import nltk
nltk.download('omw-1.4')

[nltk_data] Downloading package omw-1.4 to /Users/alina/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


True

In [44]:
# Preproecssing
# pr1 = no data preprocessing
# pre2 = replace, remove_punctionation, lower
# pre3 = replace, remove_punctionation, lower, stopword
# pre4 = replace, remove_punctionation, lower, stopword, noisy
# pre5 = replace, remove_punctionation, lower, stopword, noisy, frequency_words, scared_word
# pre6 = replace, remove_punctionation, lower, stopword, noisy, frequency_words, scared_word, lemmatization

pre1_train = preprocess_text(data_small)
pre1_test = preprocess_text(test)
pre2_train = preprocess_text(data_small, replace=1, remove_punctuation=1, lower=1)
pre2_test = preprocess_text(test, replace=1, remove_punctuation=1, lower=1)
pre3_train = preprocess_text(data_small, replace=1, remove_punctuation=1, lower=1,stopword=1)
pre3_test = preprocess_text(test, replace=1, remove_punctuation=1, lower=1,stopword=1)
pre4_train = preprocess_text(data_small, replace=1, remove_punctuation=1, lower=1,stopword=1,noisy=1)
pre4_test = preprocess_text(test, replace=1, remove_punctuation=1, lower=1,stopword=1,noisy=1)
pre5_train = preprocess_text(data_small, replace=1, remove_punctuation=1, lower=1,stopword=1,noisy=1,frequency_words=1,scared_word=1)
pre5_test = preprocess_text(test, replace=1, remove_punctuation=1, lower=1,stopword=1,noisy=1,frequency_words=1,scared_word=1)
pre6_train = preprocess_text(data_small, replace=1, remove_punctuation=1, lower=1,stopword=1,noisy=1,frequency_words=1,scared_word=1,lemmatization=1)
pre6_test = preprocess_text(test, replace=1, remove_punctuation=1, lower=1,stopword=1,noisy=1,frequency_words=1,scared_word=1,lemmatization=1)

pre_train = [(pre1_train,pre1_test),(pre2_train,pre2_test), (pre3_train,pre3_test), (pre4_train,pre4_test), (pre5_train,pre5_test),(pre6_train,pre6_test)]

for i in pre_train:
    train=i[0]
    test=i[1]
    train_tf = feature_engineering(train, tf_idf=1, train=train, word2vec=None, word_count=None)
    test_tf = feature_engineering(test, tf_idf=1, train=train, word2vec=None, word_count=None)
    clf = LogisticRegression(C=1.0, solver='lbfgs',max_iter=3000)
    clf.fit(train_tf, train['Verdict'])
    y_pred = clf.predict(test_tf)
    y_test=test['Verdict']
    score = []
    f1_macro = f1_score(y_test, y_pred, average='macro')
    accuracy = accuracy_score(y_test, y_pred)
    precision_macro = precision_score(y_test, y_pred, average='macro')
    recall_macro = recall_score(y_test, y_pred, average='macro')
    score.append([f1_macro,accuracy,precision_macro,recall_macro])
    print(score)


[nltk_data] Downloading package wordnet to /Users/alina/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package wordnet to /Users/alina/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


[[0.6564849626947871, 0.6763333333333333, 0.7131907443613872, 0.6763333333333333]]
[[0.6640057322935629, 0.6896666666666667, 0.7238802241381181, 0.6896666666666667]]
[[0.6613423276026961, 0.687, 0.7194724199670502, 0.687]]
[[0.6613423276026961, 0.687, 0.7194724199670502, 0.687]]
[[0.6613423276026961, 0.687, 0.7194724199670502, 0.687]]
[[0.6714395719827623, 0.695, 0.7302885119037538, 0.6950000000000001]]


We can find that pre6 can get the best score. (pre6 = replace, remove_punctionation, lower, stopword, noisy, frequency_words, scared_word, lemmatization)

## 5.2 Difference of feature engineering

In [19]:
pre6_train = preprocess_text(data_small, replace=1, remove_punctuation=1, lower=1,stopword=1,noisy=1,frequency_words=1,scared_word=1,lemmatization=1)
pre6_test = preprocess_text(test, replace=1, remove_punctuation=1, lower=1,stopword=1,noisy=1,frequency_words=1,scared_word=1,lemmatization=1)
# TF-IDF
train_tf = feature_engineering(pre6_train, tf_idf=1, train=pre6_train, word2vec=None, word_count=None)
test_tf = feature_engineering(pre6_test, tf_idf=1, train=pre6_train, word2vec=None, word_count=None)
#Word2Vec
train_w2v = feature_engineering(pre6_train, tf_idf=None, train=pre6_train, word2vec=1, word_count=None)
test_w2v = feature_engineering(pre6_test, tf_idf=None, train=pre6_train, word2vec=1, word_count=None)


[nltk_data] Downloading package wordnet to /Users/alina/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package wordnet to /Users/alina/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [49]:
#TF-IDF
clf = LogisticRegression(C=1.0, solver='lbfgs',max_iter=3000)
clf.fit(train_tf, pre6_train['Verdict'])
y_pred = clf.predict(test_tf)
y_test=pre6_test['Verdict']
score = []
f1_macro = f1_score(y_test, y_pred, average='macro')
accuracy = accuracy_score(y_test, y_pred)
precision_macro = precision_score(y_test, y_pred, average='macro')
recall_macro = recall_score(y_test, y_pred, average='macro')
score.append([f1_macro,accuracy,precision_macro,recall_macro])
print(score)

[[0.6714395719827623, 0.695, 0.7302885119037538, 0.6950000000000001]]


In [50]:
#Word2Vec
clf = LogisticRegression(C=1.0, solver='lbfgs',max_iter=3000)
clf.fit(train_w2v, pre6_train['Verdict'])
y_pred = clf.predict(test_w2v)
y_test=pre6_test['Verdict']
score = []
f1_macro = f1_score(y_test, y_pred, average='macro')
accuracy = accuracy_score(y_test, y_pred)
precision_macro = precision_score(y_test, y_pred, average='macro')
recall_macro = recall_score(y_test, y_pred, average='macro')
score.append([f1_macro,accuracy,precision_macro,recall_macro])
print(score)

[[0.5311481237730575, 0.5593333333333333, 0.5651206138812424, 0.5593333333333333]]


In [22]:
#LIWC
LIWC_train = pd.read_csv("./LIWC/df_train_spacy.csv")
LIWC_test = pd.read_csv("./LIWC/df_test_spacy.csv")

In [23]:
train_y = LIWC_train['Verdict']
test_y = LIWC_test['Verdict']
LIWC_train1 = LIWC_train.copy()
LIWC_test1 = LIWC_test.copy()

In [24]:
LIWC_train1.drop(columns=['Verdict','Text'],inplace=True)
LIWC_test1.drop(columns=['Verdict','Text'],inplace=True)

In [25]:
train_X = LIWC_train1
test_X = LIWC_test1

In [26]:
#Normalization
from sklearn import preprocessing

train_X_n = preprocessing.scale(train_X)
test_X_n = preprocessing.scale(test_X)

In [66]:
clf = LogisticRegression(C=1.0, solver='lbfgs',max_iter=3000)
clf.fit(train_X_n, train_y)
y_pred = clf.predict(test_X_n)
y_test=test_y
score = []
f1_macro = f1_score(y_test, y_pred, average='macro')
accuracy = accuracy_score(y_test, y_pred)
precision_macro = precision_score(y_test, y_pred, average='macro')
recall_macro = recall_score(y_test, y_pred, average='macro')
score.append([f1_macro,accuracy,precision_macro,recall_macro])
print(score)

[[0.484346249502936, 0.497, 0.4825725560197986, 0.497]]


In [27]:
#TF-IDF + LIWC

pre6_train = preprocess_text(train1, replace=1, remove_punctuation=1, lower=1,stopword=1,noisy=1,frequency_words=1,scared_word=1,lemmatization=1)
pre6_test = preprocess_text(test, replace=1, remove_punctuation=1, lower=1,stopword=1,noisy=1,frequency_words=1,scared_word=1,lemmatization=1)
# TF-IDF
train_tf_all = feature_engineering(pre6_train, tf_idf=1, train=pre6_train, word2vec=None, word_count=None)
test_tf_all = feature_engineering(pre6_test, tf_idf=1, train=pre6_train, word2vec=None, word_count=None)

[nltk_data] Downloading package wordnet to /Users/alina/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package wordnet to /Users/alina/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [37]:
from scipy.sparse import csr_matrix,hstack
train_X_n_m = csr_matrix(train_X_n)
train_X_total = hstack([train_tf_all,train_X_n_m])
test_X_n_m = csr_matrix(test_X_n)
test_X_total = hstack([test_tf_all,test_X_n_m])

In [39]:
clf = LogisticRegression(C=1.0, solver='lbfgs',max_iter=3000)
clf.fit(train_X_total, train_y)
y_pred = clf.predict(test_X_total)
y_test=test_y
score = []
f1_macro = f1_score(y_test, y_pred, average='macro')
accuracy = accuracy_score(y_test, y_pred)
precision_macro = precision_score(y_test, y_pred, average='macro')
recall_macro = recall_score(y_test, y_pred, average='macro')
score.append([f1_macro,accuracy,precision_macro,recall_macro])
print(score)

[[0.5127051017140399, 0.5283333333333333, 0.5072360453567527, 0.5283333333333333]]


## 5.3 Hyperparameters Tuning

In [25]:
C = [45,60,70,80]
result = []
for i in C:
    clf = LogisticRegression(C=i,max_iter=1000)
    clf.fit(train_tf, pre6_train['Verdict'])
    y_pred = clf.predict(test_tf)
    y_test=pre6_test['Verdict']
    f1_macro = f1_score(y_test, y_pred, average='macro')
    result.append(f1_macro)
print(result)

[0.7172891831446663, 0.7168695302792686, 0.7174449066211318, 0.7168785245858303]


In [27]:
max_iter = [100,500,1000,1500,2000,3000]
result = []
for i in max_iter:
    clf = LogisticRegression(C=40,max_iter=i)
    clf.fit(train_tf, pre6_train['Verdict'])
    y_pred = clf.predict(test_tf)
    y_test=pre6_test['Verdict']
    f1_macro = f1_score(y_test, y_pred, average='macro')
    result.append(f1_macro)
print(result)

/Users/alina/Desktop/NUS/semster4/CS5344/Project Handout and Readings/pythonProject/venv/lib/python3.8/site-packages/sklearn/linear_model/_logistic.py:814: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


[0.716091381323241, 0.7172891831446663, 0.7172891831446663, 0.7172891831446663, 0.7172891831446663, 0.7172891831446663]


In [28]:
solver = ['newton-cg','sag','saga','lbfgs']
result = []
for i in solver:
    clf = LogisticRegression(C=40,max_iter=500, solver=i)
    clf.fit(train_tf, pre6_train['Verdict'])
    y_pred = clf.predict(test_tf)
    y_test=pre6_test['Verdict']
    f1_macro = f1_score(y_test, y_pred, average='macro')
    result.append(f1_macro)
print(result)

[0.7172891831446663, 0.7164293296287682, 0.716531525532803, 0.7172891831446663]


## 5.4 Final Training

### 5.4.1 TF-IDF

In [38]:
# TF-IDF + Hyperparameter tunning

pre6_train = preprocess_text(train1, replace=1, remove_punctuation=1, lower=1,stopword=1,noisy=1,frequency_words=1,scared_word=1,lemmatization=1)
pre6_test = preprocess_text(test, replace=1, remove_punctuation=1, lower=1,stopword=1,noisy=1,frequency_words=1,scared_word=1,lemmatization=1)
train_y = pre6_train['Verdict']


[nltk_data] Downloading package wordnet to /Users/alina/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package wordnet to /Users/alina/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [40]:
X_train1, X_test1, y_train1, y_test1 = train_test_split(pre6_train,train_y,test_size=0.2, random_state=42)

In [41]:
# TF-IDF
train_tf_all = feature_engineering(X_train1, tf_idf=1, train=X_train1, word2vec=None, word_count=None)
test_tf_all = feature_engineering(X_test1, tf_idf=1, train=X_train1, word2vec=None, word_count=None)

In [42]:
train_y = X_train1['Verdict']
y_test = X_test1['Verdict']

In [76]:
# from sklearn.model_selection import GridSearchCV
lg = LogisticRegression(C=40,max_iter=1000,solver='newton-cg')
lg.fit(train_tf_all, y_train1)
y_pred = lg.predict(test_tf_all)
score = []
score2 = []
f1_macro = f1_score(y_test1, y_pred, average='macro')
accuracy = accuracy_score(y_test1, y_pred)
precision_macro = precision_score(y_test1, y_pred, average='macro')
recall_macro = recall_score(y_test1, y_pred, average='macro')
score.append([f1_macro,accuracy,precision_macro,recall_macro])
print(score)


[[0.9696072297193579, 0.9710367413775458, 0.9722313715712269, 0.9671314555427126]]


In [43]:
lg = LogisticRegression(C=40,max_iter=1000,solver='newton-cg')
lg.fit(train_tf_all, y_train1)
y_pred = lg.predict(train_tf_all)
f1_macro2 = f1_score(y_train1, y_pred, average='macro')
print(f1_macro2)

0.9999443840384346


In [36]:
# from sklearn.model_selection import GridSearchCV
lg = LogisticRegression(C=50,max_iter=1500,solver='newton-cg')
lg.fit(train_tf_all, train_y)
y_pred = lg.predict(test_tf_all)
score = []
f1_macro = f1_score(y_test, y_pred, average='macro')
accuracy = accuracy_score(y_test, y_pred)
precision_macro = precision_score(y_test, y_pred, average='macro')
recall_macro = recall_score(y_test, y_pred, average='macro')
score.append([f1_macro,accuracy,precision_macro,recall_macro])
print(score)

[[0.7489222716729435, 0.7573333333333333, 0.7737924783133107, 0.7573333333333333]]


In [37]:
# from sklearn.model_selection import GridSearchCV
lg = LogisticRegression(C=40,max_iter=1000,solver='newton-cg')
lg.fit(train_tf_all, train_y)
y_pred = lg.predict(test_tf_all)
score = []
f1_macro = f1_score(y_test, y_pred, average='macro')
accuracy = accuracy_score(y_test, y_pred)
precision_macro = precision_score(y_test, y_pred, average='macro')
recall_macro = recall_score(y_test, y_pred, average='macro')
score.append([f1_macro,accuracy,precision_macro,recall_macro])
print(score)

[[0.7493149510808985, 0.758, 0.7745434572248238, 0.758]]


### 5.4.2 TF-IDF + LIWC

In [1]:
train2 = train1.copy()
test2 = test1.copy()

NameError: name 'train1' is not defined

In [45]:
#Cross Validation
pre6_train = preprocess_text(train2, replace=1, remove_punctuation=1, lower=1,stopword=1,noisy=1,frequency_words=1,scared_word=1,lemmatization=1)
pre6_test = preprocess_text(test2, replace=1, remove_punctuation=1, lower=1,stopword=1,noisy=1,frequency_words=1,scared_word=1,lemmatization=1)
y_train = pre6_train['Verdict']
y_test = pre6_test['Verdict']


KeyboardInterrupt



In [ ]:
# TF-IDF
train_tf = feature_engineering(pre6_train, tf_idf=1, train=pre6_train, word2vec=None, word_count=None)
test_tf = feature_engineering(pre6_test, tf_idf=1, train=pre6_train, word2vec=None, word_count=None)

In [21]:
#LIWC
LIWC_train = pd.read_csv("./LIWC/df_train_spacy.csv")
LIWC_test = pd.read_csv("./LIWC/df_test_spacy.csv")

train_y = LIWC_train['Verdict']
test_y = LIWC_test['Verdict']
LIWC_train1 = LIWC_train.copy()
LIWC_test1 = LIWC_test.copy()
LIWC_train1.drop(columns=['Verdict','Text'],inplace=True)
LIWC_test1.drop(columns=['Verdict','Text'],inplace=True)
train_X = LIWC_train1
test_X = LIWC_test1
#Normalization
from sklearn import preprocessing

train_X_n = preprocessing.scale(train_X)
test_X_n = preprocessing.scale(test_X)


In [35]:
LIWC_train.shape

(45854, 114)

In [36]:
LIWC_test.shape

(3000, 114)

In [22]:
from scipy.sparse import csr_matrix,hstack
train_X_n_m = csr_matrix(train_X_n)
train_X_total = hstack([train_tf,train_X_n_m])
test_X_n_m = csr_matrix(test_X_n)
test_X_total = hstack([test_tf,test_X_n_m])

In [23]:
# from sklearn.model_selection import GridSearchCV
lg = LogisticRegression(C=0.01,max_iter=1000,solver='newton-cg')
lg.fit(train_X_total, train_y)
y_pred = lg.predict(test_X_total)
score = []
f1_macro = f1_score(y_test, y_pred, average='macro')
accuracy = accuracy_score(y_test, y_pred)
precision_macro = precision_score(y_test, y_pred, average='macro')
recall_macro = recall_score(y_test, y_pred, average='macro')
score.append([f1_macro,accuracy,precision_macro,recall_macro])
print(score)

[[0.5438564372888904, 0.5466666666666666, 0.5425634864321488, 0.5466666666666666]]


In [80]:
train2=train.copy()

In [81]:
test2 = test.copy()

In [90]:
test_X_total.shape

(9771, 844239)

In [29]:
train3 = train2.sample(frac=1.0, random_state=10)

In [30]:
train3.shape

(48854, 2)

In [46]:
train3 = train2.sample(frac=1.0, random_state=10)
pre6_train = preprocess_text(train3, replace=1, remove_punctuation=1, lower=1,stopword=1,noisy=1,frequency_words=1,scared_word=1,lemmatization=1)
# pre6_test = preprocess_text(test3, replace=1, remove_punctuation=1, lower=1,stopword=1,noisy=1,frequency_words=1,scared_word=1,lemmatization=1)
train_y = pre6_train['Verdict']
X_train = train3[0:45854]
X_test = train3[45854:48854]
y_train = X_train['Verdict']
y_test = X_test['Verdict']

# TF-IDF
train_tf = feature_engineering(X_train, tf_idf=1, train=X_train, word2vec=None, word_count=None)
test_tf = feature_engineering(X_test, tf_idf=1, train=X_train, word2vec=None, word_count=None)


#LIWC
LIWC_all = pd.read_csv("./LIWC/df_train_spacy.csv")
LIWC_all2 = LIWC_all.sample(frac=1.0, random_state=10)
LIWC_train = LIWC_all2[0:45854]
LIWC_test = LIWC_all2[45854:48854]
LIWC_train1 = LIWC_train.copy()
LIWC_test1 = LIWC_test.copy()
LIWC_y = LIWC_train['Verdict']
LIWC_y = LIWC_test['Verdict']
LIWC_train1.drop(columns=['Verdict','Text'],inplace=True)
LIWC_test1.drop(columns=['Verdict','Text'],inplace=True)
train_X = LIWC_train1
test_X = LIWC_test1

#Normalization
from sklearn import preprocessing

train_X_n = preprocessing.scale(train_X)
test_X_n = preprocessing.scale(test_X)

from scipy.sparse import csr_matrix,hstack
train_X_n_m = csr_matrix(train_X_n)
train_X_total = hstack([train_tf,train_X_n_m])
test_X_n_m = csr_matrix(test_X_n)
test_X_total = hstack([test_tf,test_X_n_m])
train_y = X_train['Verdict']
y_test = X_test['Verdict']

# from sklearn.model_selection import GridSearchCV
lg = LogisticRegression(C=0.01,max_iter=1000,solver='newton-cg')
lg.fit(train_X_total, train_y)
y_pred = lg.predict(train_X_total)
f1_macro = f1_score(train_y, y_pred, average='macro')
print(f1_macro)


# # from sklearn.model_selection import GridSearchCV
# lg = LogisticRegression(C=0.01,max_iter=1000,solver='newton-cg')
# lg.fit(train_X_total, train_y)
# y_pred = lg.predict(test_X_total)
# score = []
# f1_macro = f1_score(y_test, y_pred, average='macro')
# accuracy = accuracy_score(y_test, y_pred)
# precision_macro = precision_score(y_test, y_pred, average='macro')
# recall_macro = recall_score(y_test, y_pred, average='macro')
# score.append([f1_macro,accuracy,precision_macro,recall_macro])
# print(score)


# # TF-IDF
# train_tf = feature_engineering(pre6_train, tf_idf=1, train=pre6_train, word2vec=None, word_count=None)
# test_tf = feature_engineering(pre6_test, tf_idf=1, train=pre6_train, word2vec=None, word_count=None)

[nltk_data] Downloading package wordnet to /Users/alina/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


0.877309693722454


In [26]:
# #LIWC
# LIWC_train = pd.read_csv("./LIWC/df_train_spacy.csv")
# # LIWC_test = pd.read_csv("./LIWC/df_test_spacy.csv")
#
# train_y = LIWC_train['Verdict']
# test_y = LIWC_test['Verdict']
# LIWC_train1 = LIWC_train.copy()
# LIWC_test1 = LIWC_test.copy()
# LIWC_train1.drop(columns=['Verdict','Text'],inplace=True)
# LIWC_test1.drop(columns=['Verdict','Text'],inplace=True)
# train_X = LIWC_train1
# test_X = LIWC_test1
# #Normalization
# from sklearn import preprocessing
#
# train_X_n = preprocessing.scale(train_X)
# test_X_n = preprocessing.scale(test_X)
#
#
# # from sklearn.decomposition import PCA
# # pca = PCA(n_components=50)
# # pca_train_X = pca.fit_transform(train_X_n)
# # pca_test_X = pca.fit_transform(test_X_n)

In [32]:
from scipy.sparse import csr_matrix,hstack
train_X_n_m = csr_matrix(train_X_n)
train_X_total = hstack([train_tf,train_X_n_m])
test_X_n_m = csr_matrix(test_X_n)
test_X_total = hstack([test_tf,test_X_n_m])
train_y=train1['Verdict']
test_y = test['Verdict']

In [33]:
#Randomly choose 10% dataset from the original one
X_train, X_one_ten, y_train, y_one_ten = train_test_split(train_X_total, train_y, test_size=0.1, random_state=42)

In [31]:
C = [20.0]
result = []
for i in C:
    clf = LogisticRegression(C=i,max_iter=1000)
    clf.fit(X_one_ten, y_one_ten)
    y_pred = clf.predict(test_X_total)
    y_test=test_y
    f1_macro = f1_score(y_test, y_pred, average='macro')
    result.append(f1_macro)
print(result)

[0.41105864096998207]


In [ ]:
from sklearn.neural_network import MLPClassifier
clf = MLPClassifier(activation='logistic', alpha=0.001,learning_rate_init=0.01,early_stopping=True)
clf = clf.fit(X_one_ten,y_one_ten)
#Perform prediction
predictions = clf.predict(test_X_total)
y_test=test_y
f1_macro = f1_score(y_test, y_pred, average='macro')
result.append(f1_macro)
print(result)

## 5.5 Larger and Smaller dataset

In [57]:
#Read files
train_smaller = pd.read_csv("smaller dataset/samlltraining.csv")
test_smaller = pd.read_csv("smaller dataset/balancedtest.csv",header=None)

In [59]:
test_smaller.columns = ['Verdict','Text']
test_smaller

,Verdict,Text
0,1,When so many actors seem content to churn out ...
1,1,In what football insiders are calling an unex...
2,1,In a freak accident following Game 3 of the N....
3,1,North Koreas official news agency announced to...
4,1,The former Alaska Governor Sarah Palin would b...
...,...,...
2995,4,The Air Force mistakenly gave rival companies ...
2996,4,The United Nations climate chief on Friday cha...
2997,4,River Plate midfielder Diego Buonanotte has un...
2998,4,Lawmakers were on the brink Tuesday of exempti...


In [60]:
# TF-IDF + Hyperparameter tunning

pre6_train_s = preprocess_text(train_smaller, replace=1, remove_punctuation=1, lower=1,stopword=1,noisy=1,frequency_words=1,scared_word=1,lemmatization=1)
pre6_test_s = preprocess_text(test_smaller, replace=1, remove_punctuation=1, lower=1,stopword=1,noisy=1,frequency_words=1,scared_word=1,lemmatization=1)
# TF-IDF
train_tf_all_s = feature_engineering(train_smaller, tf_idf=1, train=train_smaller, word2vec=None, word_count=None)
test_tf_all_s = feature_engineering(test_smaller, tf_idf=1, train=train_smaller, word2vec=None, word_count=None)

[nltk_data] Downloading package wordnet to /Users/alina/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package wordnet to /Users/alina/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [61]:
train_y_s = pre6_train_s['Verdict']
y_test_s = pre6_test_s['Verdict']

In [62]:
# from sklearn.model_selection import GridSearchCV
lg = LogisticRegression(C=40,max_iter=1000,solver='newton-cg')
lg.fit(train_tf_all_s, train_y_s)
y_pred = lg.predict(test_tf_all_s)
score = []
f1_macro = f1_score(y_test_s, y_pred, average='macro')
accuracy = accuracy_score(y_test_s, y_pred)
precision_macro = precision_score(y_test_s, y_pred, average='macro')
recall_macro = recall_score(y_test_s, y_pred, average='macro')
score.append([f1_macro,accuracy,precision_macro,recall_macro])
print(score)

[[0.7132416005458351, 0.7206666666666667, 0.7387821344926073, 0.7206666666666667]]


# 6. XGBoost

In [16]:
pre6_train = preprocess_text(train1, replace=1, remove_punctuation=1, lower=1,stopword=1,noisy=1,frequency_words=1,scared_word=1,lemmatization=1)
pre6_test = preprocess_text(test, replace=1, remove_punctuation=1, lower=1,stopword=1,noisy=1,frequency_words=1,scared_word=1,lemmatization=1)
# TF-IDF
train_tf = feature_engineering(pre6_train, tf_idf=1, train=pre6_train, word2vec=None, word_count=None)
test_tf = feature_engineering(pre6_test, tf_idf=1, train=pre6_train, word2vec=None, word_count=None)

[nltk_data] Downloading package wordnet to /Users/alina/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package wordnet to /Users/alina/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [17]:
train_y = pre6_train['Verdict']
y_test = pre6_test['Verdict']

In [18]:
#Randomly choose 10% dataset from the original one
X_train, X_one_ten, y_train, y_one_ten = train_test_split(train_tf, train_y, test_size=0.1, random_state=42)

In [20]:
from sklearn.ensemble import GradientBoostingClassifier

xgb = GradientBoostingClassifier(n_estimators=100, learning_rate=1.0, max_depth=1, random_state=0)
xgb.fit(X_one_ten, y_one_ten)
y_pred = xgb.predict(test_tf)
score = []
f1_macro = f1_score(y_test, y_pred, average='macro')
accuracy = accuracy_score(y_test, y_pred)
precision_macro = precision_score(y_test, y_pred, average='macro')
recall_macro = recall_score(y_test, y_pred, average='macro')
score.append([f1_macro,accuracy,precision_macro,recall_macro])
print(score)

[[0.5623041209433175, 0.5726666666666667, 0.5799341825673994, 0.5726666666666667]]


## 6.2 Hyperparameter Tunning

In [22]:
#Learning Rate
lr = [5,10]
result = []
for i in lr:
    clf = GradientBoostingClassifier(n_estimators=100, learning_rate=i, max_depth=1, random_state=0)
    clf.fit(X_one_ten, y_one_ten)
    y_pred = clf.predict(test_tf)
    f1_macro = f1_score(y_test, y_pred, average='macro')
    result.append(f1_macro)
print(result)

[0.2355592789585601, 0.11202363899920233]


In [24]:
#Learning Rate
# n_estimators = [100,200,500]
n_estimators = [1000]
result = []
for i in n_estimators:
    clf = GradientBoostingClassifier(n_estimators=i, learning_rate=1, max_depth=1, random_state=0)
    clf.fit(X_one_ten, y_one_ten)
    y_pred = clf.predict(test_tf)
    f1_macro = f1_score(y_test, y_pred, average='macro')
    result.append(f1_macro)
print(result)

[0.5924594859660719]


In [25]:
#Max Depth
max_depth = [1,2,3,5]
result = []
for i in max_depth:
    clf = GradientBoostingClassifier(n_estimators=500, learning_rate=1, max_depth=i, random_state=0)
    clf.fit(X_one_ten, y_one_ten)
    y_pred = clf.predict(test_tf)
    f1_macro = f1_score(y_test, y_pred, average='macro')
    result.append(f1_macro)
print(result)

[0.5892325989335394, 0.5902830290613534, 0.5950372882525081, 0.6016877979796372]
